### Loading Libraries

In [1]:
%pip install Levenshtein
import numpy as np
import pandas as pd
import Levenshtein as lev
from itertools import product

Note: you may need to restart the kernel to use updated packages.


### Loading Data

In [5]:
sevMil = pd.read_csv("companies_sorted-1.csv")
ft = pd.read_csv("Forbes_top 2000_2022.csv")
db = pd.read_csv("mapping/dbpedia/data/dbpedia_preprocessed_final.csv")
#gs_ft_db = pd.read_csv("data/goldstandard/ft_db_check.csv")
#gs_ft_sevm = pd.read_csv("data/goldstandard/ft_sevM_check.csv")
gs_db_sevm = pd.read_csv("data/goldstandard/check/db_sevM_check.csv", sep = ';')

In [6]:
gs_db_sevm

,db_id,sevM_id,LevScore,db,sevMil,label
0,dbpedia_1017,7.1m_151,1.0,the coca-cola company,the coca-cola company,True
1,dbpedia_5227,7.1m_769,1.0,suncor energy,suncor energy,True
2,dbpedia_6200,7.1m_105,1.0,roche,roche,True
3,dbpedia_124,7.1m_114,1.0,halliburton,halliburton,True
4,dbpedia_2698,7.1m_1595,1.0,eastman chemical company,eastman chemical company,True
...,...,...,...,...,...,...
495,dbpedia_4029,7.1m_15,0.0,renewable energy group,citi,False
496,dbpedia_311,7.1m_1709,0.0,cemex,afpa,False
497,dbpedia_5203,7.1m_443,0.0,havertys,bd,False
498,dbpedia_135,7.1m_207,0.0,synopsys,vale,False


### Creating ID Column for Each Dataset

In [8]:
sevMil = sevMil[['name']]
sevMil = sevMil.applymap(lambda s: s.lower() if type(s) == str else s)
sevMil['id'] =  range(len(sevMil))
sevMil['id'] = "7.1m_" + sevMil['id'].astype(str)

In [9]:
ft = ft[['organizationName']]
ft = ft.applymap(lambda s: s.lower() if type(s) == str else s)
ft['id'] =  range(len(ft))
ft['id'] = "ft_id_" + ft['id'].astype(str)

In [10]:
dbpedia = db[['name']]
dbpedia = dbpedia.applymap(lambda s: s.lower() if type(s) == str else s)
dbpedia['id'] =  range(len(db))
dbpedia['id'] = "dbpedia_" + dbpedia['id'].astype(str)

#### Between FT and DBPedia Datasets

In [6]:
# Calculate the distance (ratio)
ft_db_lev = pd.DataFrame(product(ft['organizationName'], dbpedia['name']), columns=["ft","dbpedia"])
# Since there are duplicate name
ft_db_lev = ft_db_lev.drop_duplicates()
ft_db_lev["LevScore"] = ft_db_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs 
ft_db_lev_filtered = ft_db_lev[ft_db_lev['LevScore'] > 0.8]
ft_db_lev_false = ft_db_lev[ft_db_lev['LevScore'] < 0.3]

# Identify matching and similiar-but-unmatching pairs 
confirmed_true_ftdb = ft_db_lev_filtered[ft_db_lev_filtered['LevScore'] == 1]
edge_cases_ftdb = ft_db_lev_filtered[ft_db_lev_filtered['LevScore'] != 1]
false_cases_ftdb = ft_db_lev_false

# Filter the combinations already in goldstandard (used as test set in machine learning)
keys = ['ft', 'dbpedia']
gs_keys = gs_ft_db.set_index(keys).index
true_keys = confirmed_true_ftdb.set_index(keys).index
edge_keys = edge_cases_ftdb.set_index(keys).index
false_keys = false_cases_ftdb.set_index(keys).index

confirmed_true_ftdb = confirmed_true_ftdb[~true_keys.isin(gs_keys)]
edge_cases_ftdb = edge_cases_ftdb[~edge_keys.isin(gs_keys)]
false_cases_ftdb = false_cases_ftdb[~false_keys.isin(gs_keys)]

# Sample 160 for matching cases, 240 for corner cases, 400 for non-matching cases (total = 800)
true_sample = confirmed_true_ftdb.sample(160, random_state=1)
edge_sample = edge_cases_ftdb.sample(240, random_state=1)
false_sample = false_cases_ftdb.sample(400, random_state=1)

# Goldstandard output
gold_standard_ft_db = pd.concat([true_sample, edge_sample, false_sample])
gold_standard_ft_db = gold_standard_ft_db.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ft_db.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db.merge(dbpedia, how = 'left', left_on = 'dbpedia', right_on = 'name')
gold_standard_ft_db.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_ft_db = gold_standard_ft_db[['ft_id', 'db_id', 'LevScore', 'ft', 'dbpedia']]
gold_standard_ft_db['label'] = gold_standard_ft_db.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ft_db = gold_standard_ft_db.drop_duplicates(subset=['ft', 'dbpedia'], keep='first')
gold_standard_ft_db

,ft_id,db_id,LevScore,ft,dbpedia,label
0,ft_id_959,dbpedia_6248,1.000000,voya financial,voya financial,True
1,ft_id_920,dbpedia_845,1.000000,m&g,m&g,True
2,ft_id_99,dbpedia_929,1.000000,united parcel service,united parcel service,True
3,ft_id_1378,dbpedia_6773,1.000000,extra space storage,extra space storage,True
4,ft_id_1684,dbpedia_4567,1.000000,strabag,strabag,True
...,...,...,...,...,...,...
865,ft_id_1890,dbpedia_3840,0.242424,pennymac financial services,endava,False
866,ft_id_512,dbpedia_5433,0.270270,murata manufacturing,ineos styrolution,False
867,ft_id_1156,dbpedia_4864,0.250000,yum china holdings,bank of cyprus,False
868,ft_id_1788,dbpedia_4505,0.232558,smoore international holdings,bsh hausgeräte,False


In [7]:
gold_standard_ft_db.describe(include = 'all')

,ft_id,db_id,LevScore,ft,dbpedia,label
count,800,800,800.000000,800,800,800
unique,608,713,NaN,608,713,2
top,ft_id_1163,dbpedia_1836,NaN,cj corporation,op financial group,False
freq,8,6,NaN,8,6,640
mean,NaN,NaN,0.554445,NaN,NaN,NaN
std,NaN,NaN,0.359001,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,0.216216,NaN,NaN,NaN
50%,NaN,NaN,0.551994,NaN,NaN,NaN
75%,NaN,NaN,0.875000,NaN,NaN,NaN


#### Between FT and 7Mil datasets

In [20]:
# Calculate the distance (ratio)
ft_sevM_lev = pd.DataFrame(product(ft['organizationName'], sevMil['name']), columns=["ft","sevMil"])
# Since there are duplicate name, plan to deal with it in Java
ft_sevM_lev = ft_sevM_lev.drop_duplicates()
ft_sevM_lev["LevScore"] = ft_sevM_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs, adjust LevScore to 0.75 for more possible corner cases, as is 0.8 not enought for 240 cases 
ft_sevM_lev_filtered = ft_sevM_lev[ft_sevM_lev['LevScore'] > 0.75]
ft_sevM_lev_false = ft_sevM_lev[ft_sevM_lev['LevScore'] < 0.3]

# Identify matching and similiar-but-unmatching pairs
confirmed_true_ftsevM = ft_sevM_lev_filtered[ft_sevM_lev_filtered['LevScore'] == 1]
edge_cases_ftsevM = ft_sevM_lev_filtered[ft_sevM_lev_filtered['LevScore'] != 1]
false_cases_ftsevM = ft_sevM_lev_false


# Filter the combinations already in goldstandard (used as test set in machine learning)
keys = ['ft', 'sevMil']
gs_keys = gs_ft_sevm.set_index(keys).index
true_keys = confirmed_true_ftsevM.set_index(keys).index
edge_keys = edge_cases_ftsevM.set_index(keys).index
false_keys = false_cases_ftsevM.set_index(keys).index

confirmed_true_ftsevM = confirmed_true_ftsevM[~true_keys.isin(gs_keys)]
edge_cases_ftsevM = edge_cases_ftsevM[~edge_keys.isin(gs_keys)]
false_cases_ftsevM = false_cases_ftsevM[~false_keys.isin(gs_keys)]

# Sample 160 for matching cases, 240 for corner cases, 400 for non-matching cases (total = 800)
true_sample = confirmed_true_ftsevM.sample(160, random_state=1)
edge_sample = edge_cases_ftsevM.sample(240, random_state=1)
false_sample = false_cases_ftsevM.sample(400, random_state=1)

# Goldstandard output
gold_standard_ftsevM = pd.concat([true_sample, edge_sample, false_sample])
gold_standard_ftsevM = gold_standard_ftsevM.merge(ft, how='left', left_on='ft', right_on='organizationName')
gold_standard_ftsevM.rename(columns={'id':'ft_id'}, inplace = True)
gold_standard_ftsevM = gold_standard_ftsevM.merge(sevMil, how = 'left', left_on = 'sevMil', right_on = 'name')
gold_standard_ftsevM.rename(columns={'id':'sevM_id'}, inplace = True)
gold_standard_ftsevM = gold_standard_ftsevM[['ft_id', 'sevM_id', 'LevScore', 'ft', 'sevMil']]
gold_standard_ftsevM['label'] = gold_standard_ftsevM.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_ftsevM = gold_standard_ftsevM.drop_duplicates(subset=['ft', 'sevMil'], keep='first')
gold_standard_ftsevM


,ft_id,sevM_id,LevScore,ft,sevMil,label
0,ft_id_1587,7.1m_123,1.000000,dxc technology,dxc technology,True
1,ft_id_348,7.1m_375,1.000000,texas instruments,texas instruments,True
2,ft_id_967,7.1m_593,1.000000,quest diagnostics,quest diagnostics,True
3,ft_id_261,7.1m_769,1.000000,suncor energy,suncor energy,True
4,ft_id_432,7.1m_872,1.000000,national bank of canada,national bank of canada,True
...,...,...,...,...,...,...
798,ft_id_1486,7.1m_850,0.125000,thai beverage,adt,False
799,ft_id_1770,7.1m_614,0.258065,bank of jiujiang,coca-cola femsa,False
800,ft_id_499,7.1m_1138,0.111111,vonovia,icici prudential life insurance company limited,False
801,ft_id_95,7.1m_423,0.181818,citic,walt disney world,False


In [21]:
gold_standard_ftsevM.describe(include = 'all')

,ft_id,sevM_id,LevScore,ft,sevMil,label
count,800,800,800.000000,800,800,800
unique,619,608,NaN,619,608,2
top,ft_id_1219,7.1m_796,NaN,carnival corporation,colliers international,False
freq,7,9,NaN,7,9,640
mean,NaN,NaN,0.534125,NaN,NaN,NaN
std,NaN,NaN,0.350032,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,0.214286,NaN,NaN,NaN
50%,NaN,NaN,0.526714,NaN,NaN,NaN
75%,NaN,NaN,0.800000,NaN,NaN,NaN


#### Between db and 7Mil datasets

In [14]:
# Calculate the distance (ratio)
db_sevM_lev = pd.DataFrame(product(dbpedia['name'], sevMil['name']), columns=["db","sevMil"])
# Since there are duplicate name, plan to deal with it in Java
db_sevM_lev = db_sevM_lev.drop_duplicates()
db_sevM_lev["LevScore"] = db_sevM_lev.apply(lambda x: lev.ratio(x[0],x[1]), axis=1)

# Filter similiar pairs, adjust LevScore to 0.75 for more possible corner cases, as is 0.8 not enought for 240 cases 
db_sevM_lev_filtered = db_sevM_lev[db_sevM_lev['LevScore'] > 0.75]
db_sevM_lev_false = db_sevM_lev[db_sevM_lev['LevScore'] < 0.3]

# Identify matching and similiar-but-unmatching pairs
confirmed_true_dbsevM = db_sevM_lev_filtered[db_sevM_lev_filtered['LevScore'] == 1]
edge_cases_dbsevM = db_sevM_lev_filtered[db_sevM_lev_filtered['LevScore'] != 1]
false_cases_dbsevM = db_sevM_lev_false


# Filter the combinations already in goldstandard (used as test set in machine learning)
keys = ['db', 'sevMil']
gs_keys = gs_db_sevm.set_index(keys).index
true_keys = confirmed_true_dbsevM.set_index(keys).index
edge_keys = edge_cases_dbsevM.set_index(keys).index
false_keys = false_cases_dbsevM.set_index(keys).index

confirmed_true_dbsevM = confirmed_true_dbsevM[~true_keys.isin(gs_keys)]
edge_cases_dbsevM = edge_cases_dbsevM[~edge_keys.isin(gs_keys)]
false_cases_dbsevM = false_cases_dbsevM[~false_keys.isin(gs_keys)]

# Sample 160 for matching cases, 240 for corner cases, 400 for non-matching cases (total = 800)
true_sample = confirmed_true_dbsevM.sample(160, random_state=1)
edge_sample = edge_cases_dbsevM.sample(240, random_state=1)
false_sample = false_cases_dbsevM.sample(400, random_state=1)

# Goldstandard output
gold_standard_dbsevM = pd.concat([true_sample, edge_sample, false_sample])
gold_standard_dbsevM = gold_standard_dbsevM.merge(dbpedia, how='left', left_on='db', right_on='name')
gold_standard_dbsevM.rename(columns={'id':'db_id'}, inplace = True)
gold_standard_dbsevM = gold_standard_dbsevM.merge(sevMil, how = 'left', left_on = 'sevMil', right_on = 'name')
gold_standard_dbsevM.rename(columns={'id':'sevM_id'}, inplace = True)
gold_standard_dbsevM = gold_standard_dbsevM[['db_id', 'sevM_id', 'LevScore', 'db', 'sevMil']]
gold_standard_dbsevM['label'] = gold_standard_dbsevM.apply(lambda x: True if x['LevScore'] == 1 else False, axis = 1)
gold_standard_dbsevM = gold_standard_dbsevM.drop_duplicates(subset=['db', 'sevMil'], keep='first')
gold_standard_dbsevM


,db_id,sevM_id,LevScore,db,sevMil,label
0,dbpedia_5248,7.1m_234,1.000000,boehringer ingelheim,boehringer ingelheim,True
1,dbpedia_4447,7.1m_976,1.000000,technipfmc,technipfmc,True
2,dbpedia_6421,7.1m_257,1.000000,aetna,aetna,True
3,dbpedia_3096,7.1m_552,1.000000,rite aid,rite aid,True
4,dbpedia_6158,7.1m_204,1.000000,fedex express,fedex express,True
...,...,...,...,...,...,...
877,dbpedia_5781,7.1m_449,0.250000,snc-lavalin,promsvyaznedvizhimost,False
878,dbpedia_212,7.1m_1442,0.242424,red lobster,johns hopkins hospital,False
879,dbpedia_5615,7.1m_905,0.166667,p. c. richard & son,coles,False
880,dbpedia_474,7.1m_452,0.176471,beacon building products,teksystems,False


Printing data for manually check

In [22]:
gold_standard_ft_db.sort_values('LevScore', ascending = False).to_csv('data/goldstandard/train_ft_db_check.csv', index=False)
gold_standard_ftsevM.sort_values('LevScore', ascending = False).to_csv('data/goldstandard/train_ft_sevM_check.csv', index=False)

In [15]:
gold_standard_dbsevM.sort_values('LevScore', ascending = False).to_csv('data/goldstandard/check/train_db_sevm_check.csv', index=False)


Printing Output